In [1]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate gradio

In [2]:
import os
import requests
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
import gradio as gr

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

!pip install -U bitsandbytes

In [3]:
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
  model_name,
  device_map="auto",
  quantization_config=quant_config
)

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [14]:
def generate_dataset(topic, number_of_data, inst1, resp1, inst2, resp2, inst3, resp3):
    # Convert user inputs into multi-shot examples
    multi_shot_examples = [
        {"instruction": inst1, "response": resp1},
        {"instruction": inst2, "response": resp2},
        {"instruction": inst3, "response": resp3}
    ]

    # System prompt
    system_prompt = f"""
    You are a helpful assistant whose main purpose is to generate datasets.
    Topic: {topic}
    Return the dataset in JSON format. Use examples with simple, fun, and easy-to-understand instructions for beginners.
    Include the following examples: {multi_shot_examples}
    Return {number_of_data} examples each time.
    Do not repeat the provided examples.
    """

    # Example Messages
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Please generate my dataset for {topic}"}
    ]

    # Tokenize Input
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
    streamer = TextStreamer(tokenizer)

    # Generate Output
    outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

    # Decode and Return
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


def gradio_interface(topic, number_of_data, inst1, resp1, inst2, resp2, inst3, resp3):
    return generate_dataset(topic, number_of_data, inst1, resp1, inst2, resp2, inst3, resp3)

In [15]:
default_topic = "Talking to people about Gen Ai, what it means at the basic level and expand."
default_number_of_data = 10
default_multi_shot_examples = [
    {
        "instruction": "What would be the benefit of AI?",
        "response": "It is helping people retrive information faster"
    },
    {
        "instruction": "How should i use AI and ML?",
        "response": "start by identifying specific tasks where they can provide value, such as content generation, data analysis, or automation"
    },
    {
        "instruction": "why should I use AI and LLM?",
        "response": "enhancing efficiency, creativity, and knowledge accessibility"
    }
]

In [16]:
gr_interface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Textbox(label="Topic", value=default_topic, lines=2),
        gr.Number(label="Number of Examples", value=default_number_of_data, precision=0),
        gr.Textbox(label="Instruction 1", value=default_multi_shot_examples[0]["instruction"]),
        gr.Textbox(label="Response 1", value=default_multi_shot_examples[0]["response"]),
        gr.Textbox(label="Instruction 2", value=default_multi_shot_examples[1]["instruction"]),
        gr.Textbox(label="Response 2", value=default_multi_shot_examples[1]["response"]),
        gr.Textbox(label="Instruction 3", value=default_multi_shot_examples[2]["instruction"]),
        gr.Textbox(label="Response 3", value=default_multi_shot_examples[2]["response"]),
    ],
    outputs=gr.Textbox(label="Generated Dataset")
)

In [17]:
gr_interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4e290f150c720693d7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
